# Predict NP Connectivity from Single-Cell Transcriptomes

In [ ]:
# Important to keep this in sync whenever renaming this notebook
notebook_name = 'NP_Nets'

# Import modules
import pymap as pm # SJS module: pymap.py must reside in current directory
import json
import shutil
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcol
import matplotlib.patches as patches
import seaborn as sns
from matplotlib.patches import Rectangle, Polygon
from matplotlib.backends.backend_pdf import PdfPages

# Assemble the general filepaths
name_root = notebook_name
results_path = 'Products/'+name_root+'/'
pdf_path = results_path+'PDFs/'
csv_path = results_path+'CSVs/'
zip_path = results_path+'ZIPs/'
temp_path = 'Temp/'

# Make some choices...
dataset = 'Y21ss'
#dataset = 'Y23'
#dataset = 'H19' 
#geneset_list = ['30NPPs','52NPRs','30NPPs*Rs'] # '60NPPs', '30NPPs', '30NPPs*Rs','52NPRs', '77NPRs' or '137NPs'
row_sort = 'SN' # 'SN' (none), 'SE' (each), 'SR' (sort "regions" only: apply same order to remainder)
# Adjust minimum group size according to dataset choice
#min_tally = 20 if 'ss' in dataset else 100

if dataset == 'Y21ss' or dataset == 'H19': # SMART-Seq
    min_tally = 10
    threshold = 10
else: # TenX
    min_tally = 100
    threshold = 1

if dataset[0] == 'Y': # Path for tables specific to species but not to dataset
    species = 'mouse'
    tables_path = 'Resources/Mouse_tables/'
else:
    species = 'human'
    tables_path = 'Resources/Human_tables/'    

# Read annotated expression data
source_path = 'Resources/'+dataset+'_nMod_counts.csv.zip'
support_path = 'Resources/'+dataset+'_nMod_support.json'
#source_path = 'Resources/'+dataset+'_NP_counts.csv.zip'
#support_path = 'Resources/'+dataset+'_NP_support.json'
print('Reading',source_path)
anno_counts = pd.read_csv(source_path)

# Read support dictionary
with open(support_path, 'r') as fp:
    support_dict = json.load(fp)

# Extract annotation value dictionaries
region_colors = dict(zip(anno_counts['region'],anno_counts['region_color']))
class_colors = dict(zip(anno_counts['class'],anno_counts['class_color']))
class_colors = {key: class_colors[key] for key in sorted(class_colors)}
subclass_colors = dict(zip(anno_counts['subclass'],anno_counts['subclass_color']))
subclass_colors = {key: subclass_colors[key] for key in sorted(subclass_colors)}
subclass_classes = dict(zip(anno_counts['subclass'],anno_counts['class']))
subclass_classes = {key: subclass_classes[key] for key in sorted(subclass_classes)}
subclass_shorts = dict(zip(anno_counts['subclass'],anno_counts['subclass']))
subclass_shorts = {key: subclass_shorts[key].split(' ',1)[1] for key in sorted(subclass_shorts)}
cluster_colors = dict(zip(anno_counts['cluster'],anno_counts['cluster_color']))
cluster_colors = {key: cluster_colors[key] for key in sorted(cluster_colors)}

# Read CSV table to form region dictionaries
df = pd.read_csv('Resources/'+dataset+'_region_table.csv')
region_names = df.set_index('region_label')['region_full'].to_dict()
region_shorts = df.set_index('region_label')['region_short'].to_dict()
region_longs = {value: key for key, value in region_shorts.items()} # inverse of region_shorts
region_zones = df.set_index('region_short')['zone'].to_dict()

# Form annotation unique value lists
types = (list(set(anno_counts['type']))); types.sort()
regions = list(region_colors.keys())
classes = list(class_colors.keys())
subclasses = list(subclass_colors.keys())
clusters = list(cluster_colors.keys())

if dataset == 'Y23': # 'supertypes' in Y23 dataset only
    supertype_colors = dict(zip(anno_counts['supertype'],anno_counts['supertype_color']))
    supertype_colors = {key: supertype_colors[key] for key in sorted(supertype_colors)}
    supertypes = list(supertype_colors.keys())
    major_annos = ['type','region','class','subclass','supertype','cluster']

if dataset[0:3] == "Y21": # 'neighborhood' in Y21 datasets only
    neighborhood_colors = dict(zip(anno_counts['neighborhood'],anno_counts['neighborhood_color']))
    neighborhood_colors = {key: neighborhood_colors[key] for key in sorted(neighborhood_colors)}
    neighborhoods = list(neighborhood_colors.keys())
    major_annos = ['type','region','class','subclass','cluster']
    
if dataset == 'H19': # No neighborhoods, no supertypes
    major_annos = ['type','region','class','subclass','cluster']

# Read CSV table to form gene label dictionary
#df = pd.read_csv('Resources/All_nMods_labeled.csv', usecols=[1,2], index_col=0)
df = pd.read_csv(tables_path + 'All_nMods_labeled.csv')
gene_labels = df.to_dict()['Label']

# Configure multiindex
all_annos = support_dict['annotation_labels']
minor_annos = [s for s in all_annos if s not in major_annos]
anno_counts.drop(columns=minor_annos, inplace=True)
anno_counts.set_index(major_annos, inplace=True)

# Sanity check
if species == 'mouse':
    print('Sanity check:\n'+str(anno_counts[['Cck','Npy','Vip','Cckbr','Kng1','Bdkrb1']].sum()))
    
else:
    print('Sanity check:\n'+str(anno_counts[['CCK','NPY','VIP','CCKBR','KNG1','BDKRB1']].sum()))


# Read in a list of NPP genes and extend matrix with aggregated conjugate NP-GPCR proxy genes

geneset = '30NPPs' #'30NPPs'

# Read a list of NPP genes
genelist = list(pd.read_csv(tables_path+geneset+'.csv')['Symbol'])   
counts = anno_counts[genelist].copy()

# Convert an NPP matrix to an aggregated, conjugate NP-GPCR matrix
assert 'NPP' in geneset, 'This cell requires an NPP gene list!'

# Read conjugate pair table and use to generate aggregates of NP_GPCRs conjugates to the NPPs
# Read pair table
#pair_table = pd.read_csv('Resources/Mouse_NP_pairs.csv')
pair_table = pd.read_csv(tables_path + 'NP_pairs.csv')
pair_table['Pair Label'] = pair_table['NPP'] + '->' + pair_table['NP-GPCR']
pair_id = list(pair_table['Pair ID'])
pair_npps = list(pair_table['NPP'])
pair_nprs = list(pair_table['NP-GPCR'])
pair_labels = list(pair_table['Pair Label'])
pair_table.set_index('Pair ID', inplace=True)
# Build cognate NPP/aggregate-NP-GPCRs dictionary
df = pair_table[['NPP','NP-GPCR']].set_index('NPP').copy()
df['Aggs'] = df['NP-GPCR']+' '
df = df.groupby('NPP').sum()
aggs_dict = df['Aggs'].to_dict()
for npp in aggs_dict: aggs_dict[npp] = aggs_dict[npp].split()
# Build dataframe with aggregated NP-GPCRs conjugate to included NPPs
npp_list = genelist
df = anno_counts[npp_list].copy()
for npp in npp_list: df[npp+'Rs'] = anno_counts[aggs_dict[npp]].sum(axis=1)
npr_list = [x+'Rs' for x in npp_list]
counts = df.copy()

#geneset = geneset+'+'+newlist_name
genelist = npp_list+npr_list

print('Genelist:',geneset,', Counts matrix shape:', counts.shape)


In [ ]:
counts.loc['01 Neuronal'].to_csv('Y21_30NPPs_30Cognate-Aggs.csv')

In [ ]:
# Plot NPP x NP-GPCR subclass outer product matrices by region, write results to a PDF and a CSV

def form(x): # Format with pretty decimal places control
    return ('{:,.02f}'.format(x) if mat_max < 20 else '{:,.0f}'.format(x))

#NPPs = ['Cck'] #,'Vip','Penk']
#NPPs = ['Vip','Npy','Tac2'] #,'Pomc','Npy','Penk','Sst','Cort','Pnoc','Pdyn']
NPPs = npp_list
#NPPs = ['Penk','Pdyn','Pomc','Pnoc','Tac1','Cck','Npy','Sst'] # For Y23 STRv and STRd
#NPPs = ['Sst'] #,'Cck','Adcyap1','Vip','Grp','Trh'] # For Y21 ALM and HIP
#NPPs = ['Cck','Npy','Sst','Vip','Adcyap1','Grp','Trh','Penk','Pdyn','Pomc','Pnoc','Tac1'] # Consensus
#NPPs = ['Penk','Pdyn','Tac1']
#NPPs = ['Sst','Cck','Adcyap1','Vip','Grp','Trh','Penk','Pdyn','Tac1']

REGIONs = regions
if dataset[0:3] == "Y21": REGIONs.remove('014 CLA') # only one cell...
#REGIONs = ['001 VISp','012 ALM','028 HIP'] #Y21
#REGIONs = ['012 ALM']#,'028 HIP'] #Y21
#REGIONs = ['001 MOp','018 STRd','019 STRv'] #,'014 HIP','026 P'] #Y23
#REGIONs = ['001 MOp'] #Y23
#REGIONs = ['018 STRd'] #Y23
#REGIONs = ['019 STRv'] #Y23

metric = 'fracpos'
#metric = 'mean'

#if dataset == 'Y21ss': threshold = 10
#if dataset == 'Y21tenX': threshold = 1 
#if dataset == 'Y23': threshold = 1

title_points = 28
ax_points = 32
num_points =28
corner_points = 36
tally_points = 28
tick_points = 26
    
sns.set_theme(font_scale = 1.4, rc={'patch.linewidth': 0})

# Get a serial number and save a copy of this notebook
sn = pm.serial(5)
source_notebook_path = notebook_name+'.ipynb'
destination_notebook_path = 'Notebooks/'+notebook_name+'_'+sn+'.ipynb'
shutil.copy2(source_notebook_path, destination_notebook_path)

# Form handle string
handle = dataset+'_Nets_'+metric+'_'
if len(NPPs) < 4:
    for s in NPPs: handle += s
else: handle += 'manyNPPs'
handle += '_'
if len(REGIONs) < 4:
    for s in REGIONs: handle += region_shorts[s]
else: handle += 'manyRegs' 
handle += '_'+sn

# Form file paths
pdf_file = pdf_path+handle+'.pdf'
csv_file = csv_path+handle+'.csv'
zip_file = zip_path+handle+'.zip'
temp_csv = temp_path+'temp.csv'

# Intialize PDF writing
pdf = PdfPages(pdf_file)

# Map regions, write results to a PDF and a CSV
summary = pd.DataFrame()
summary_inx = 0
for npp_label in NPPs:
    # Set up useful slices
    npr_label = npp_label+'Rs'   
    idx = pd.IndexSlice
    df = counts.loc[idx[types,regions,classes,subclasses], :]
    df = df.loc['01 Neuronal']
    #df = df.loc['02 Glial']
    df = df.groupby(['region','subclass']).filter(lambda x: len(x) >= min_tally) # filter out small groups
    reg_subclasses = list(set(df.reset_index()['subclass']))
    reg_subclasses.sort()
    means_groups = df.groupby(['region','subclass'])
    df = (df >= threshold).astype(int)
    pos_groups = df.groupby(['region','subclass'])
    
    for region in REGIONs:
        # Derive useful taxon lists
        df = means_groups.count().iloc[:,0].T.loc[region]
        cell_tally_list = list(df.map('{:,.0f}'.format)) # numbers of cells
        subclass_longs = df.index
        class_longs = [subclass_classes[s] for s in subclass_longs]
        shorts = [subclass_shorts[s] for s in subclass_longs]
        mat_size = len(df)

        # Compute expression mean count dataframe
        df = means_groups.mean().loc[region]
        npp_means = df[npp_label]
        npr_means = df[npr_label]
        df = np.outer(npp_means, npr_means)
        df = pd.DataFrame(df, index=shorts, columns=shorts)
        means_outer = df.copy()
        
        # Compute positive cell expression fraction dataframe
        df = pos_groups.mean().loc[region]                
        npp_fracpos = df[npp_label]
        npr_fracpos = df[npr_label]
        df = np.outer(npp_fracpos, npr_fracpos)
        df = pd.DataFrame(df, index=shorts, columns=shorts)
        cells_pos_outer = df.copy()
                
        # Plot means or fraction_pos
        if metric == 'mean':
            df = np.sqrt(means_outer)
            title = dataset + ' sn:' + sn
            cbar_label = '\nSqrt(Product of Counts)'
            cbar_kws={'shrink':0.4,'pad':0.15,'aspect':15, 'label':cbar_label}
            label_units = 'mean counts per subtype'
            new_row = npr_means.to_frame().T
            new_col = npp_means.to_frame()
        elif metric == 'fracpos':
            df = 100*cells_pos_outer
            title = dataset + ' thr='+str(threshold)+' sn:'+sn
            cbar_kws={'shrink':0.4,'pad':0.15,'aspect':15,'label':'\nDoubly Positive Cells'}
            label_units = 'fraction postive per subtype'
            new_row = 100*npr_fracpos.to_frame().T
            new_col = 100*npp_fracpos.to_frame()
        else: raise Exception("Something went wrong!")

        npr_max = new_row.max().max()
        npr_mean = new_row.mean().mean()
        npr_min = new_row.min().min()
        npp_max = new_col.max().max()
        npp_mean = new_col.mean().min()
        npp_min = new_col.min().min()  
        mat_max = df.max().max()
        mat_mean = df.sum().sum()/(mat_size**2)
        mat_min = df.min().min()
        diag_max = max(np.diag(df.values))
        diag_mean = sum(np.diag(df.values))/len(df)
        diag_min = min(np.diag(df.values))
        if mat_max > 0.0001: df = df / mat_max

        new_row_max = new_row.max().max()
        if new_row_max > 0.0001: new_row = new_row / new_row_max
        new_col_max = new_col.max().max()
        if new_col_max > 0.0001: new_col = new_col / new_col_max

        # Prepend spacer-separated row and column vectors        
        new_rows = new_row.T
        new_rows.insert(1,'',[np.nan] * len(new_row.T))
        new_rows = new_rows.T
        new_rows.columns = shorts
        df = pd.concat([new_rows, df])
        df.insert(0,'spacer',[np.nan] * len(df))
        df.insert(0,'npp',[np.nan]*2 + list(new_col[npp_label]))
        df = df.rename(columns={'spacer': '', 'npp': ''}, index={npr_label: ''})

        # Set frame display metrics
        frame = df
        frame_size = len(frame)
        figure_size = (1+frame_size/2,1+frame_size/2)
        fig, ax = plt.subplots(1,1, figsize=figure_size) 
        print('figure_size =', figure_size[0], 'x', figure_size[1])        

        ax = sns.heatmap(df, ax=ax, square=True, xticklabels=1, yticklabels=1, cmap='rocket',
                         vmin=0, vmax=1, cbar=False) # cbar_kws=cbar_kws)
        ax.collections[0].cmap.set_bad(color='white')
        
        secax_x = ax.secondary_xaxis(0.0)
        secax_x.xaxis.set_label_position('bottom')
        
        # Form frame labeling strings
        units = '' if metric == 'mean' else '%'
        # strings displayed automatically sns.heatmap
        title += '\n'+region_names[region]+' ('+region_shorts[region]+')'
        ax.set_title(title, pad=16, fontsize=title_points)
        ax.set_xlabel(npr_label+' receptor vector', x=(frame_size+2)/(2*frame_size), labelpad=16, fontsize=ax_points)
        ax.set_ylabel(npp_label+' modulator vector', y=(frame_size-2)/(2*frame_size),  labelpad=16, fontsize=ax_points)
        # strings not handled automatically by sns.heatmap
        secax_x.set_xlabel('number of cells', x=(frame_size+2)/(2*frame_size), labelpad=16, fontsize=ax_points)
        corner = npp_label+' → '+npr_label+'\nmodulatory\nnetwork graph\nprediction'
        numerics = (npr_label+' vector\n'+
                    'max: '+form(npr_max)+units+'\n'+
                    'mean: '+form(npr_mean)+units+'\n\n'+
                    npp_label+' vector\n'+
                    'max: '+form(npp_max)+units+'\n'+
                    'mean: '+form(npp_mean)+units+'\n\n'+
                    npp_label+' x '+npr_label+'\n'+
                    'outer product\n'+
                    'max: '+form(mat_max)+units+'\n'+
                    'mean: '+form(mat_mean)+units+'\n\n'+
                    'diagonal\n'+
                    'max: '+form(diag_max)+units+'\n'
                    'mean: '+form(diag_mean)+units)
        
        # Display labeling strings not handled automatically by sns.heatmap
        ax.text(1.05, 0, numerics, transform=ax.transAxes, color='k', ha='left', va='bottom',fontsize=num_points)
        ax.text(0, 1, corner, transform=ax.transAxes, color='k', ha='right', va='bottom', fontsize=corner_points)
        
        # Find tick pad size
        tick_pad_points = 2 * 72 * figure_size[0] / frame_size

        # Label left axis with NPP taxa
        labels = list(df.index)
        ax.tick_params(axis='y', length=0, labelrotation=0, pad=tick_pad_points)
        ax.set_yticks([s+0.6 for s in list(range(0,len(labels)))],labels=labels,fontsize=tick_points)
        ax.yaxis.set_label_position('left')

        # Label top axis with NP-GPCR taxa
        labels = list(df.columns)    
        ax.tick_params(axis='x', length=0, labelrotation=90, pad=tick_pad_points)
        ax.set_xticks([s+0.59 for s in list(range(0,len(labels)))],labels=labels,fontsize=tick_points)
        ax.xaxis.tick_top()
        ax.xaxis.set_label_position('top')
        
        # Label bottom axis ticks with cell tallies
        secax_x.set_xticks(np.arange(0.5, len(labels)+0.5), labels=['','']+cell_tally_list, ha='center')
        secax_x.tick_params(axis='x', length=0, pad=16, labelrotation=90,labelsize=tick_points)

        # Find class boundary line positions
        lines = []; zone = []
        for i, subclass in enumerate(subclass_longs):
            if subclass_classes[subclass] != zone: lines += [i]; zone = subclass_classes[subclass]
        lines += [mat_size]
        lines = [s+2 for s in lines]

        color = 'blue'
        lw = 4 # 0.3 * frame_size     
        # Draw class boundaries on product matrix      
        ax.hlines(lines, 2, mat_size+2, colors=color, lw=lw).set_clip_on(False)
        ax.vlines(lines, 2, mat_size+2, colors=color, lw=lw).set_clip_on(False)

        tail = 3
        # Draw class boundaries and edges on row and column vectors
        ax.hlines([0,1], 2, frame_size, colors=color, lw=lw).set_clip_on(False)
        ax.vlines([0,1], 2, frame_size, colors=color, lw=lw).set_clip_on(False)
        ax.hlines(lines, 1, -tail, colors=color, lw=lw).set_clip_on(False)
        ax.vlines(lines, 1, -tail, colors=color, lw=lw).set_clip_on(False)
        ax.plot([2,mat_size+2], [1,1], color=color, lw=lw, clip_on=False)
        ax.plot([1,1], [2,mat_size+2], color=color, lw=lw, clip_on=False)

        # Draw autocrines diagonal
        ax.plot([2,mat_size+2], [2,mat_size+2], color='cyan', lw=2*lw, ls=':', clip_on=False)
        
        # Draw class color patches outside left and top axis margins
        colors = [class_colors[s] for s in class_longs]
        #w = mat_size/len(colors)
        for inx in [0,1]:
            for i, color in enumerate(colors):
                # Draw top color patches
                patch = Polygon([(i+2,-1), (i+2,-2), (i+3,-2), (i+3,-1), (i+2,-1)], facecolor=color, zorder=0)
                ax.add_patch(patch).set_clip_on(False)
                # Draw left color patches
                patch = Polygon([(-1,i+2), (-2,i+2), (-2,i+3), (-1,i+3), (-1,i+2)], facecolor=color, zorder=0)
                ax.add_patch(patch).set_clip_on(False)

        # Draw subclass color patches outside left and top axis margins
        colors = [subclass_colors[s] for s in subclass_longs]
        for inx in [0,1]:
            for i, color in enumerate(colors):
                # Draw top color patches
                patch = Polygon([(i+2,0), (i+2,-1), (i+3,-1), (i+3,0), (i+2,0)], facecolor=color, zorder=0)
                ax.add_patch(patch).set_clip_on(False)
                # Draw left color patches
                patch = Polygon([(0,i+2), (-1,i+2), (-1,i+3), (0,i+3), (0,i+2)], facecolor=color, zorder=0)
                ax.add_patch(patch).set_clip_on(False)

        # Update summary table
        data = {'npp_label':npp_label, 'npr_label':npr_label, 'region':region, 'subclasses': mat_size,
                'mat_max':mat_max, 'diag_max':diag_max, 'npp_max': npp_max, 'npr_max': npr_max,
                'mat_mean':mat_mean, 'diag_mean':diag_mean, 'npp_mean': npp_mean, 'npr_mean': npr_mean,
                'mat_min':mat_min, 'diag_min':diag_min, 'npp_min': npp_min, 'npr_min': npr_min}
        summary = pd.concat([summary, pd.DataFrame(data,index=[summary_inx])])
        summary_inx += 1
        
        pdf.savefig(dpi=300, transparent=True, bbox_inches='tight', pad_inches=0.2)
        plt.show()
        print(npp_label,'page written to:', pdf_file)

pdf.close()
    
summary.to_csv(csv_file); print('Summary table written to CSV:', csv_file)
